# Pytorch基本操作考察

In [2]:
import torch

## 基本操作实验1
使用 𝐓𝐞𝐧𝐬𝐨𝐫 初始化一个 𝟏×𝟑 的矩阵 𝑴 和一个 𝟐×𝟏 的矩阵 𝑵，对两矩阵进行减法操作（要求实现三种不同的形式），给出结果并分析三种方式的不同（如果出现报错，分析报错的原因），同时需要指出在计算过程中发生了什么

计算过程中发生了什么？\
广播操作

使用第三种减法操作会报错，因为是原地操作，当进行广播后，计算结果和原矩阵的形状不同，所以会报错

In [3]:
M = torch.empty(1, 3) # M 1 x 3
N = torch.empty(2, 1) # N 1 x 2
print('M:{}\nN:{}'.format(M, N))
# 减法1
print(M - N)
# 减法2
print(torch.sub(M, N))

M:tensor([[0., 0., 0.]])
N:tensor([[8.4078e-45],
        [0.0000e+00]])
tensor([[-8.4078e-45, -8.4078e-45, -8.4078e-45],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00]])
tensor([[-8.4078e-45, -8.4078e-45, -8.4078e-45],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00]])


In [4]:
# 减法3--有问题(inplace原地操作)
print(M.sub_(N))

RuntimeError: output with shape [1, 3] doesn't match the broadcast shape [2, 3]

## 基本操作实验2
1) 利用 𝐓𝐞𝐧𝐬𝐨𝐫 创建两个大小分别 𝟑×𝟐 和 𝟒×𝟐 的随机数矩阵 𝑷 和 𝑸 ，要求服从均值为0，标准差0.01为的正态分布 \
2) 对第二步得到的矩阵 𝑸 进行形状变换得到 𝑸 的转置 $𝑸^𝑻$  \
3) 对上述得到的矩阵 𝑷 和矩阵 $𝑸^𝑻$ 求内积

In [5]:
# 创建tensor
P = torch.normal(0, 0.01, size=(3, 2))
Q = torch.normal(0, 0.01, size=(4, 2))
print('P:{}\nQ:{}'.format(P, Q))

P:tensor([[-0.0176,  0.0069],
        [ 0.0043,  0.0066],
        [-0.0086,  0.0211]])
Q:tensor([[ 9.3939e-05, -2.1687e-03],
        [-1.1382e-02,  7.4087e-03],
        [ 1.2412e-02,  1.9412e-02],
        [-9.8564e-03,  1.0435e-02]])


In [6]:
# Q转置
Q_t = Q.T
Q_t

tensor([[ 9.3939e-05, -1.1382e-02,  1.2412e-02, -9.8564e-03],
        [-2.1687e-03,  7.4087e-03,  1.9412e-02,  1.0435e-02]])

In [13]:
# 矩阵运算P和Q.t
torch.mm(P, Q_t)

tensor([[-1.6598e-05,  2.5122e-04, -8.4493e-05,  2.4525e-04],
        [-1.3915e-05, -2.0185e-07,  1.8176e-04,  2.6368e-05],
        [-4.6632e-05,  2.5487e-04,  3.0293e-04,  3.0564e-04]])

## 基本操作实验3
给定公式$𝑦_3=𝑦_1+𝑦_2=𝑥^2+𝑥^3$，且 $𝑥=1$。利用学习所得到的Tensor的相关知识，求$𝑦_3$对的梯度𝑥，即$𝑑𝑦_3$/$𝑑𝑥$。要求在计算过程中，在计算 $𝑥^3$ 时中断梯度的追踪，观察结果并进行原因分析

$x^3$的梯度是2而不是5的原因：\
由于 $y_2$的定义是被torch.no_grad():包裹的，所以与$y_2$有关的梯度是不会回传的，只有与$y_1$有关的梯度才会回传，即$x^2$对 $x$的梯度。\
而y2.requires_grad=False，所以不能调用 y2.backward()，会报错

In [8]:
# 根据题目给的已知进行初始化
x = torch.ones(1, requires_grad=True)
y1 = torch.pow(x, 2)
# 中断x^3的梯度追踪
with torch.no_grad():
    y2 = torch.pow(x, 3)
y3 = y1 + y2
x.requires_grad, y1.requires_grad, y2.requires_grad, y3.requires_grad

(True, True, False, True)

In [9]:
# o1来记录y1
o1 = y1.sum()
o1.backward(retain_graph=True)
x.grad

tensor([2.])

In [10]:
# o3来记录y3
o3 = y3.sum()
# x.grad.zero_()  # 如果没有清零会有叠加
o3.backward(retain_graph=True)
x.grad

tensor([4.])

In [11]:
# 调用禁用的变量y2
o2 = y2.sum()
o2.backward()
# 出错了

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn